In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
from scipy.stats import chi2_contingency
import xlrd as xlrd
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingRandomSearchCV 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [31]:
# Load data
data= pd.read_csv(r'diabetic_data.csv')
data

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide_metformin,glipizide_metformin,glimepiride_pioglitazone,metformin_rosiglitazone,metformin_pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [32]:
# Create a table of the explanatory variables with their characteristics: Type(categorical/continous),string/number and number 
# of unique values
col = [c for c in data.columns if c not in ['encounter_id','patient_nbr','readmitted']]
variable_type=[]
string_ind=[]
numclasses=[]

for c in col:
    numclasses.append(len(np.unique(data[[c]])))
    
    if c in ['admission_type_id','discharge_disposition_id','admission_source_id']:
        variable_type.append('categorical')
        string_ind.append('Number')
        
        
    elif (is_string_dtype(np.array(data[[c]])) == True):
        variable_type.append('categorical')
        string_ind.append('String')
        
    elif (len(np.unique(data[[c]])) <= 10) and is_string_dtype(np.array(data[[c]])) == False:
        variable_type.append('categorical')
        string_ind.append('Number')    
        
    else: 
        variable_type.append('continous')
        string_ind.append('Number')

In [33]:
columns = [e for e in data.columns.to_list() if e not in ['encounter_id','patient_nbr','readmitted']]
df = pd.DataFrame(list(zip(columns,variable_type,string_ind, numclasses)),
               columns =['column_name','variable_type','String_Number_type','number_of_values'])

In [35]:
df


,column_name,variable_type,String_Number_type,number_of_values
0,race,categorical,String,6
1,gender,categorical,String,3
2,age,categorical,String,10
3,weight,categorical,String,10
4,admission_type_id,categorical,Number,8
5,discharge_disposition_id,categorical,Number,26
6,admission_source_id,categorical,Number,17
7,time_in_hospital,continous,Number,14
8,payer_code,categorical,String,18
9,medical_specialty,categorical,String,73


In [36]:
# Find range for continous variables
num_var_list = df.loc[df['variable_type']=='continous','column_name']

max_val=[]
min_val=[]
for c in num_var_list:
    max_val.append(data[c].max())
    min_val.append(data[c].min())

num_var_range = pd.DataFrame(list(zip(num_var_list,min_val,max_val)),
               columns =['column_name','min','max'])
num_var_range

,column_name,min,max
0,time_in_hospital,1,14
1,num_lab_procedures,1,132
2,num_medications,1,81
3,number_outpatient,0,42
4,number_emergency,0,76
5,number_inpatient,0,21
6,number_diagnoses,1,16


In [37]:
# Build ordinal variables from continous variables by dividing continous variables into quartiles
num_var_q_list =[]
for var in num_var_list:
    #data[var+'_'+'q'] = pd.qcut(data[var],4,labels=False)
    cut_series, cut_intervals = pd.qcut(data[var],4,retbins=True,duplicates='drop',labels=False)
    data[var+'_'+'q'] = cut_series.astype(str)
    num_var_q_list.append(var+'_'+'q')

In [38]:
# This step of data cleaning was added retrospectively after looking at the univariate table, built in the next step, and
# observing the following values representing missing data
data_clean = data.replace(['?','Unknown/Invalid'], np.nan)

In [39]:
## univariate analysis 
# The univariate analysis includes two outputs:
# Univariate table - in which the percentage of readmission is calculated for each of the variables categories.  
# pvalues_table - contains the P-values of each variable for predicting readmission.     

# Create a list of explanatory variables
cat_var_list_ = df.loc[df['variable_type']=='categorical','column_name']
filter_list = ['admission_type_id','discharge_disposition_id','admission_source_id','diag_1','diag_2','diag_3','payer_code','medical_specialty']
cat_var_list = [x for x in cat_var_list_ if x not in filter_list]
var_list = cat_var_list+num_var_q_list
var_list

# Univariate analysis
univariate_table = pd.DataFrame(columns = ['readmitted','<30', '>30','NO']) 
pvalues = []
for var in var_list:
    
    univariate_table_ = pd.crosstab(data_clean[var],data_clean['readmitted'],normalize=True)

    univariate_table_['var_name'] = [var]*univariate_table_.shape[0]
    univariate_table = pd.concat([univariate_table, univariate_table_])
    
    stat, p, dof, expected = chi2_contingency(pd.crosstab(data_clean[var], columns=data_clean['readmitted']))
    pvalues.append(p)
    
univariate_table
pvalues_table = pd.DataFrame(pvalues, columns = ['pvalue'])
pvalues_table['var_name'] = var_list



In [40]:
#writer = pd.ExcelWriter(r'C:\Users\liatw\OneDrive\Documents\menora\results_21_11.xlsx', engine='xlsxwriter')
#univariate_table.to_excel(writer, sheet_name='univariate_table')
#pvalues_table.to_excel(writer, sheet_name='pvalues')
#writer.close()

In [41]:
# Find significant variables from univariate analysis
signif_vars = pvalues_table.loc[pvalues_table['pvalue'] < 0.05,'var_name'] 
signif_vars

0                     race
1                   gender
2                      age
3                   weight
4           num_procedures
5            max_glu_serum
6                A1Cresult
7                metformin
8              repaglinide
11             glimepiride
13               glipizide
16            pioglitazone
17           rosiglitazone
18                acarbose
24                 insulin
30                  change
31             diabetesMed
32      time_in_hospital_q
33    num_lab_procedures_q
34       num_medications_q
37      number_inpatient_q
38      number_diagnoses_q
Name: var_name, dtype: object

In [42]:
# Enumerate ordinal significant ordinal variables 
age_mapper = [{'[0-10)': 0, '[10-20)': 1,'[20-30)': 2, '[30-40)': 3, '[40-50)': 4,'[50-60)': 5,
                       '[60-70)': 6,'[70-80)': 7,'[80-90)': 8,'[90-100)': 9}]

data_clean["age_o"] = data_clean['age'].replace(age_mapper)

weight_mapper = [{'?': np.nan, '[0-25)': 0,'[25-50)': 1, '[50-75)': 2, '[75-100)': 3,'[100-125)': 4,
                       '[125-150)': 5,'[150-175)': 6,'[175-200)': 7,'>200': 8}]
data_clean["weight_o"] = data_clean['weight'].replace(weight_mapper)

max_glu_mapper = [{'None': 0, 'Norm': 1,'>200': 2, '>300': 3}]
data_clean["max_glu_o"] = data_clean['max_glu_serum'].replace(max_glu_mapper)

A1Cresult_mapper = [{'None': 0, 'Norm': 1,'>7': 2, '>8': 3}]
data_clean["A1Cresult_o"] = data_clean['A1Cresult'].replace(A1Cresult_mapper)

In [44]:
# Create dummy variables from significant categorical variables 
encoder = OneHotEncoder(sparse=False)
var_dummy = encoder.fit_transform(data_clean[['race','gender','change','diabetesMed','metformin','repaglinide','glimepiride',
                                          'glipizide','pioglitazone','rosiglitazone','acarbose','insulin']])
var_dummy
var_model_=pd.DataFrame(var_dummy, columns=encoder.get_feature_names_out())

# Create the final table with the explanatory variables for the model  
#var_quantile = data_clean[['time_in_hospital_q','num_lab_procedures_q','num_medications_q','number_diagnoses_q']]
var_ordered = data_clean[['age_o','weight_o','max_glu_o','A1Cresult_o','num_procedures','time_in_hospital_q',
                          'num_lab_procedures_q','num_medications_q','number_diagnoses_q']]

var_model = pd.concat((var_model_, var_ordered), axis=1)

# Encode the dummy variable
le = LabelEncoder()
target_encoded = le.fit_transform(data_clean['readmitted'])
target_encoded

array([2, 1, 2, ..., 2, 2, 2])

In [45]:
var_model

,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,race_nan,gender_Female,gender_Male,gender_nan,change_Ch,...,insulin_Up,age_o,weight_o,max_glu_o,A1Cresult_o,num_procedures,time_in_hospital_q,num_lab_procedures_q,num_medications_q,number_diagnoses_q
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,[0-10),NaN,None,None,0,0,1,0,0
1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,[10-20),NaN,None,None,0,1,3,2,2
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,[20-30),NaN,None,None,5,0,0,1,0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,[30-40),NaN,None,None,1,0,1,2,1
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,[40-50),NaN,None,None,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,[70-80),NaN,None,>8,0,1,2,2,2
101762,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,[80-90),NaN,None,None,3,2,1,2,2
101763,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,[70-80),NaN,None,None,0,0,2,0,3
101764,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,[80-90),NaN,None,None,2,3,2,3,2


In [116]:
###########################################################
### Building predictive model using Random Forest ###
###########################################################

X_train, X_test, y_train, y_test = train_test_split(var_model,target_encoded , test_size=0.3, random_state=0)
model=RandomForestClassifier(n_estimators=1000,max_features='sqrt')
fitted_model=model.fit(X_train,y_train) 
predictions=fitted_model.predict(X_test)
metrics.accuracy_score(y_test, predictions, normalize=True)


0.49554536521454307

In [100]:
# Optimize the model 
random_forest_model = RandomForestClassifier(n_jobs=-1,max_features='sqrt')
param_grid= {"n_estimators": [10,100,500,1000],
             "max_depth": [1,5,10,15],
             "min_samples_leaf": [1,2,4,10,15,30,50]}
#grid_search = GridSearchCV(estimator=random_forest_model,param_grid=param_grid,cv=10)
#grid_search.fit(X_train,y_train)
#print(grid_search.best_params_)
search = HalvingRandomSearchCV(random_forest_model, param_grid,
                                resource='n_samples',
                                max_resources='auto',
                                random_state=0).fit(var_model, target_encoded)


C:\Users\liatw\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 112 is smaller than n_iter=3392. Running 112 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [101]:
print(search.best_params_)
predictions = search.predict(X_test)
print(metrics.accuracy_score(y_test, predictions))

{'n_estimators': 1000, 'min_samples_leaf': 1, 'max_depth': 1}
0.5380281690140845
